# Computerphysik Programmiertutorial 6
Prof. Dr. Matteo Rizzi und Dr. Markus Schmitt - Institut für Theoretische Physik, Universität zu Köln
&nbsp;

**ILIAS**: [https://www.ilias.uni-koeln.de/ilias/goto_uk_crs_3862489.html](https://www.ilias.uni-koeln.de/ilias/goto_uk_crs_3862489.html)

**Github**: [https://github.com/markusschmitt/compphys2021](https://github.com/markusschmitt/compphys2021)

**Inhalt dieses Notebooks**: Jonglieren mit Arrays, Matrix-Diagonalisierung (Anwendungsbeispiel), dünne Matrizen, [E] Zusammengesetzte Datentypen: `struct`s


## Jonglieren mit Arrays



Dimensionen ändern: `reshape`

Eine einzelne Spalte extrahieren

Eine einzelne Zeile extrahieren

Eine Untermatrix extrahieren

Indizes vom Ende zählen:

## Anwendungsbeispiel Matrix-Diagonalisierung

### Nicht-lineare Funktionen von Matrizen

Nicht-lineare Funktionen von Matrizen wie z.B. $exp(\cdot)$ oder $log(\cdot)$ sind über die entsprechende Taylorreihe definiert. So ist z.B.

$$
exp(A)=\sum_n\frac{1}{n!}A^n
$$

Wenn $A$ diagonalisierbar ist, können wir das Berechnen hoher Potenzen der Matrix vermeiden. Mit der Zerlegung $A=VDV^{-1}$ bekommen wir 

$$
A^n=\big(VDV^{-1}\big)^n=VD^nV^{-1}
$$

wobei $D$ die Diagonalmatrix mit den Eigenwerten $d_j$ auf der Diagonalen ist. Dadurch vereinfacht sich das Problem, denn das Potenzieren einer Diagonalmatrix entspricht dem Potenzieren der Diagonaleinträge. Durch Einsetzen in die Taylorreihe bekommen wir

$$
exp(A)=\sum_n\frac{1}{n!}VD^nV^{-1}=V\bigg(\sum_n\frac{1}{n!}D^n\bigg)V^{-1}
=V
\begin{pmatrix}\exp(d_1)&&0\\&\ddots&\\0&&\exp(d_N)\end{pmatrix}
V^{-1}
$$

Diese Konstruktion ist immer Möglich, wenn eine Funktion als Taylorreihe geschrieben werden kann.

### Beispiel aus der Quantenmechanik

Die zeitabhängige Schrödingergleichung

$$
-i\frac{d}{dt}\psi = H~\psi
$$

hat die formale Lösung

$$
\psi(t) = \underbrace{\exp(-iHt)}_{=U_H(t)}\psi(t=0)
$$

Dabei ist der Hamilton-Operator $H$ ein linearer Operator auf dem Hilbertraum $\mathcal H$, der die Wellenfunktion $\psi$ beherbergt. Falls $\mathcal H$ endlich-dimensional ist, ist $H$ also eine Matrix. Ein sehr grundlegendes Beispiel ist ein magnetisches Teilchen, das sich in einem externen Magnetfeld befindet. Im einfachsten Fall ist der Hilbertraum zweidimensional, also $\psi\in\mathbb C^2$. Wenn ein Magnetfeld in Richtung 

$$
\vec B=B\begin{pmatrix}\cos\phi\sin\theta\\\sin\phi\sin\theta\\\cos\theta\end{pmatrix}
$$

angelegt wird, ist die zugehörige Schrödingergleichung

$$
-i\frac{d}{dt}\begin{pmatrix}\psi_1\\\psi_2\end{pmatrix}
=
\begin{pmatrix}
B\cos\theta&B(\cos\phi-i\sin\phi)\sin\theta\\
B(\cos\phi+i\sin\phi)\sin\theta&-B\cos\theta
\end{pmatrix}
\begin{pmatrix}\psi_1\\\psi_2\end{pmatrix}
$$

In [ ]:
phi=0.2*pi
theta=0.123*pi
B = 1.0

H = B * [cos(theta) (cos(phi)-1.0im*sin(phi))*sin(theta); (cos(phi)+1.0im*sin(phi))*sin(theta) -cos(theta)]

psi_0 = [1.0, 0.0]

U(H,4.3)*psi_0

Nach den Regeln der Quantenmechanik ist die Polarisierung unseres magnetischen Teilchens in $z$-Richtung gegeben durch $M_z=|\psi_1|^2-|\psi_2|^2$. Die Zeitentwicklung dieser Polarisierung können wir uns nun anschauen:

In [ ]:
using PyPlot

times = collect(0:0.1:10)
magnetizations = Float64[]

for t in times
    psi_t = U(H,t)*psi_0
    
    magn = real(psi_t[1]*conj(psi_t[1])-psi_t[2]*conj(psi_t[2]))
    
    push!(magnetizations, magn)
end

plot(times, magnetizations)
xlabel("Zeit")
ylabel(L"$z$-Polarisierung");

## Dünne Matrizen

Matrizen, die in physikalischen Anwendungen interessant sind, haben oft viel Struktur. In verschiednene Anwendungen tauchen z.B. Matrizen auf, die nur wenige von Null verschiedene Einträge haben -- sogenannte **dünn besetzte Matrizen**. In solchen Fällen ist es günstig nicht die komplette Matrix im Speicher abzulegen, sondern nur die wenigen Einträge, die nicht Null sind.

Beispiel:

$$
M=\begin{pmatrix}5&0&0&0\\0&8&0&0\\0&0&7&3\\0&4&0&1\end{pmatrix}
$$

Um solche Matrizen effizient zu speichern gibt es verschiedene Datenstrukturen. Die Idee ist immer, dass es reicht die Werte der nicht-Null Einträge zu speichern, sowie ihre jeweilige Position in der Matrix.

Eine Möglichkeit ist das **Compressed Sparse Column (CSC)** Format. In diesem Format werden drei Arrays verwendet um die dünne Matrix zu Beschreiben:

1. Das erste Array enthält die **Werte** der nicht-Null Matrixeinträge entsprechend ihrer *spaltenweise* Reihenfolge:
```
values = [5, 8, 4, 7, 3, 1]
```
2. Das zweite Array enthält zugehörigen **Zeilenindizes** in der gleichen Reihenfolge:
```
row_idx = [1, 2, 4, 3, 3, 4]
```
3. Das dritte Array enthält die sogenannten **Spaltenzeiger**. Die Spaltenzeiger sind die Indizes des ersten Eintrags jeder Spalte im `values` bzw. `row_idx` Array. Das Array enthält außerdem immer einen zusätzlichen letzten Eintrag, der der Zahl der Einträge in `values` plus 1 entspricht:
```
col_ptr = [1, 2, 4, 5, 7]
```

Außerdem speichern wir die Dimension der Matrix, also $m$ und $n$ für eine $m\times n$-Matrix.

Eine solche Datenstruktur können wir zum Beispiel als Dictionary implementieren:

Um die Funktionsweise zu illustrieren, implementieren wir ein Matrix-Vektor-Produkt für unsere dünne Matrix:

In [ ]:
function sparse_mvp(A, x)
    # Array für Ergebnis anlegen
    res = zeros(A["n_rows"])
    
    # zur Übersichtlichkeit
    col_ptr = A["col_ptr"]
    row_idx = A["row_idx"]
    values = A["values"]
    
    # Schleife für das Matrix-Vektor-Produkt
    for j in 1:(size(col_ptr)[1]-1)
        col_range = col_ptr[j]:col_ptr[j+1]-1 # Bereich der aktuellen Spalte im values Array
        row_indices = row_idx[col_range]      # Zeilen-Indizes, die zur aktuellen Spalte gehören
        
        res[row_indices] += values[col_range] * x[j]
    end
    
    return res
end

In [ ]:
dense_M = float([5 0 0 0; 0 8 0 0; 0 0 7 3; 0 4 0 1])

dense_M * mein_vektor

Julia stellt standardmäßig eine Datenstruktur für dünne Matrizen im CSC-Format bereit (im Standardpaket `SparseArrays`, [Dokumentation](https://docs.julialang.org/en/v1/stdlib/SparseArrays/)):

Dünne Matrizen können damit auf unterschiedliche Weise erzeugt werden.

Entweder aus einer dichten Matrix:

Oder durch Übergeben der Zeilenindizes, Spaltenindizes, und Werte aller Einträge

Genau wie bei dichten Matrizen können wir einzelene Einträge auslesen ...

... und ändern

Das Matrix-Vektor-Produkt ist mit dieser Datenstruktur auch bereits implementiert:

Genauso gibt es dünne Vektoren:

## [E] Zusammengesetzte Datentypen: `struct`s

Julia bietet die Möglichkeit eigene Zusammengesetzte Datentypen als `struct`s zu definieren ([Dokumentation](https://docs.julialang.org/en/v1/manual/types/#Composite-Types)).

In [ ]:
struct MySparse
    n_rows::Int
    n_cols::Int
    values::Vector{Float64}
    row_idx::Vector{Int}
    col_ptr::Vector{Int}
end

In [ ]:
mat = MySparse(4, 4, float([5, 8 ,4 , 7, 3, 1]), [1, 2, 4, 3, 3, 4], [1, 2, 4, 5, 7])

Unser neuer zusammengesetzter Datentyp ist Julia jetzt bekannt:

Auf die einzelnen Felder können wir mit `.<Feldname>` zugreifen:

Damit können wir unser dünnes Matrix-Vektor-Produkt umschreiben (*multiple dispatch*):

In [ ]:
function sparse_mvp(A::MySparse, x)
    # Array für Ergebnis anlegen
    res = zeros(A.n_rows)
    
    # Schleife für das Matrix-Vektor-Produkt
    for j in 1:(size(A.col_ptr)[1]-1)
        col_range = A.col_ptr[j]:A.col_ptr[j+1]-1 # Bereich der aktuellen Spalte im values Array
        row_indices = A.row_idx[col_range]        # Zeilen-Indizes, die zur aktuellen Spalte gehören
        
        res[row_indices] += A.values[col_range] * x[j]
    end
    
    return res
end

sparse_mvp(mat, mein_vektor)

**Hinweis:** `struct`s sind *immutable*:

Das kann mit `mutable struct`s umgangen werden:

In [ ]:
mutable struct MyMutableSparse
    n_rows::Int
    n_cols::Int
    values::Vector{Float64}
    row_idx::Vector{Int}
    col_ptr::Vector{Int}
end

In [ ]:
mat_mutable = MyMutableSparse(4, 4, float([5, 8, 4, 7, 3, 1]), [1, 2, 4, 3, 3, 4], [1, 2, 4, 5, 7])

mat_mutable.n_rows=5